In [3]:
from nltk import word_tokenize, pos_tag

In [4]:
#libabries Data Transformation

import pandas as pd
import numpy as np
import json as json

import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
#Daten Import 
#TITEL INSIDE KORRIGIEREN

df_new = pd.read_json("/Users/jonathan/Desktop/data diss/ABOUTDATA.json")
df_new = df_new.append(pd.read_json("/Users/jonathan/Desktop/data diss/MEDICATION.json"))
df_new = df_new.append(pd.read_json("/Users/jonathan/Desktop/data diss/SYMPTOMS.json"))

df_new

,Documents
0,{'Post': 'Hi Crazyguy and welcome the the boar...
1,{'Post': 'To my knowledge he is not an MS spec...
2,{'Post': 'Hi Crazyguy. I feel like I’m talking...
3,{'Post': 'Do they no longer use spinal taps to...
4,"{'Post': 'Hi Wendy, it's nice to see you posti..."
...,...
326,{'Post': 'I have never used it for my jaw but ...
327,{'Post': 'Originally posted by Barrister16 Vie...
328,{'Post': 'Tongue problems here too Hello! I ha...
329,"{'Post': 're: Botox Hello, I have suffered wit..."


In [6]:
#Richtiges Format 

bn = pd.DataFrame(df_new.values.tolist())[0]
dft= pd.DataFrame.from_records(bn)

dft

,Post,Titel,Date,User,Number_of_total_posts
0,Hi Crazyguy and welcome the the boards! Sounds...,Sort of new here,"04-06-2019, 01:29 PM",Seasha,4492
1,To my knowledge he is not an MS specialist. No...,Sort of new here,"04-06-2019, 04:52 PM",Crazyguy,31
2,Hi Crazyguy.\nI feel like I’m talking to a wal...,Sort of new here,"04-06-2019, 06:04 PM",SNOOPY,4237
3,Do they no longer use spinal taps to see if th...,Sort of new here,"04-06-2019, 08:21 PM",TheBeans,54
4,"Hi Wendy, it's nice to see you posting again\n...",Sort of new here,"04-06-2019, 09:43 PM",SNOOPY,4237
...,...,...,...,...,...
2395,I have never used it for my jaw but I do use i...,Botox for spasticity in jaw and mouth?,"01-14-2015, 01:20 AM",Barrister16,36
2396,Originally posted by Barrister16 View Post\nI ...,Botox for spasticity in jaw and mouth?,"01-14-2015, 06:09 AM",rdmc,2568
2397,Tongue problems here too\nHello!\nI have had s...,Botox for spasticity in jaw and mouth?,"08-13-2015, 02:06 PM",Grade1,27
2398,"re: Botox\nHello,\nI have suffered with jaw/mo...",Botox for spasticity in jaw and mouth?,"08-26-2015, 06:49 PM",mlissa67,141


In [7]:
#Auswahl der relevanten Spalten Post+Titel (Titel pro Post aufgenommen) TITEL FEHLT

dfpost = dft["Post"]

dfpost



0       Hi Crazyguy and welcome the the boards! Sounds...
1       To my knowledge he is not an MS specialist. No...
2       Hi Crazyguy.\nI feel like I’m talking to a wal...
3       Do they no longer use spinal taps to see if th...
4       Hi Wendy, it's nice to see you posting again\n...
                              ...                        
2395    I have never used it for my jaw but I do use i...
2396    Originally posted by Barrister16 View Post\nI ...
2397    Tongue problems here too\nHello!\nI have had s...
2398    re: Botox\nHello,\nI have suffered with jaw/mo...
2399    Hi Grade1 and Mlissa,\nI just saw this thread ...
Name: Post, Length: 2400, dtype: object

In [8]:
#Textcleaning
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'[‘’“”…]', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub(r'\n', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [9]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(dfpost.apply(round1))
data_cleanStrAll = data_clean["Post"].apply(str)


data_clean
data_cleanStrAll
#dataframe['column_name'].apply(str)




0       hi crazyguy and welcome the the boards sounds ...
1       to my knowledge he is not an ms specialist not...
2       hi crazyguyi feel like im talking to a wall wh...
3       do they no longer use spinal taps to see if th...
4       hi wendy its nice to see you posting againorig...
                              ...                        
2395    i have never used it for my jaw but i do use i...
2396    originally posted by  view posti have never us...
2397    tongue problems here toohelloi have had stiffn...
2398    re botoxhelloi have suffered with jawmouth pai...
2399    hi  and mlissai just saw this thread was resur...
Name: Post, Length: 2400, dtype: object

In [679]:
#POS Tagger NN and JJ
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    
    #######noun und adjektive##########################
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    #nur noun
    #is_noun_adj = lambda pos: pos[:2] == 'NN' 
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [680]:
# Extrahiere nur NN oder NN + JJ
data_nouns_adj = pd.DataFrame(data_cleanStrAll.apply(nouns_adj))
data_nouns_adj

,Post
0,hi crazyguy boards answers conditions limbs ot...
1,knowledge ms specialist sure area close hes em...
2,hi crazyguyi im wall i possibility im diagnost...
3,spinal taps damage nerves spinal column
4,hi nice thebeans spinal taps damage nerves spi...
...,...
2395,i jaw i muscle spasms neck head upper i inject...
2396,view posti jaw i muscle spasms neck head upper...
2397,tongue problems stiffness tongue ten years mai...
2398,re botoxhelloi jawmouth pain april month i ms ...


In [681]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

#Topic model mit skelearn 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

#Tokenizer
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text) if(len(word) >2 and len(word.strip("Xx/")) >2)]
    #stems = [stemmer.stem(item) for item in tokens]
    return tokens

[nltk_data] Downloading package punkt to /Users/jonathan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [778]:
#Unigram = NN , bei NN als bigram entsteht viel noise
#bigram = NN JJ
keywords = df_new = pd.read_json("/Users/jonathan/Desktop/data diss/uni und bigramm no Intervention.json") 
#keywords = df_new = pd.read_json("/Users/jonathan/Desktop/data diss/uni und bigramm.json")
#keywords = df_new = pd.read_json("/Users/jonathan/Desktop/data diss/10000biNNJJ dd09clean.json")
#keywords = df_new = pd.read_json("/Users/jonathan/Desktop/data diss/features2000NNuni,dd08clean.json")
keywords = pd.DataFrame(keywords)

keywords

,0
0,lot pain
1,pain legs
2,leg pain
3,nerve pain
4,brain lesions
...,...
480,neurodegeneration
481,lymph
482,influenza
483,asthma


In [779]:
#Features müssen in einer iterierbaren Liste sein

lst=[]
for kw in keywords [0]:
    x = kw
    lst.append(x)



featrue = pd.DataFrame(lst)

featrue

,0
0,lot pain
1,pain legs
2,leg pain
3,nerve pain
4,brain lesions
...,...
480,neurodegeneration
481,lymph
482,influenza
483,asthma


In [780]:
#Liste cleanen

featuresallein = pd.DataFrame(featrue[0])
featuresallein

#als list um voc in lda zu platzieren

ListeFeature = featuresallein.values.tolist()

ListeFeature


flat_list_feature = []
for sublist in ListeFeature:
    for item in sublist:
        flat_list_feature.append(item)
        
len(flat_list_feature)

485

In [781]:
# Document Term Matrix mit eigenen Keywords
#ngram range anpassen an Keywords 1 =1,2 2=2,3
my_vocabulary= flat_list_feature
vectorizer = CountVectorizer(ngram_range=(1,2),max_df=0.7,min_df=1)
df=vectorizer.fit_transform(my_vocabulary) #Learn the vocabulary dictionary and return document-term matrix.
tf = vectorizer.transform(data_nouns_adj) #Transform documents to document-term matrix.




In [782]:
(vectorizer.vocabulary_) 

{'lot': 278,
 'pain': 368,
 'lot pain': 282,
 'legs': 255,
 'pain legs': 386,
 'leg': 244,
 'leg pain': 249,
 'nerve': 334,
 'nerve pain': 335,
 'brain': 49,
 'lesions': 264,
 'brain lesions': 51,
 'lesions brain': 265,
 'flare': 154,
 'symptoms': 526,
 'flare symptoms': 156,
 'spinal': 494,
 'cord': 82,
 'spinal cord': 495,
 'neuro': 346,
 'neuro pain': 347,
 'chronic': 71,
 'chronic pain': 73,
 'fatigue': 131,
 'pain fatigue': 378,
 'eye': 121,
 'eye pain': 122,
 'spasticity': 485,
 'side': 466,
 'spasticity side': 492,
 'relief': 447,
 'pain relief': 395,
 'brain spinal': 53,
 'active': 3,
 'active lesions': 4,
 'left': 238,
 'left leg': 243,
 'problems': 426,
 'problems legs': 431,
 'pain side': 397,
 'spasms': 481,
 'leg spasms': 252,
 'symptoms fatigue': 530,
 'arm': 18,
 'arm leg': 19,
 'pains': 405,
 'pains leg': 408,
 'worse': 577,
 'pain worse': 401,
 'normal': 357,
 'normal pain': 359,
 'spine': 496,
 'brain spine': 54,
 'psychological': 436,
 'symptoms psychological': 536,


In [783]:
tf.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [784]:

#assymerisch oder symmetrisch
#wenig topics pro docukoment oder viele?
#wenig wörter pro topic oder viele


#lda = decomposition.LatentDirichletAllocation(n_components=20,max_iter=50,learning_method='online',learning_offset=50,n_jobs=1,random_state=111)
#anzahl topics über components 
#ganz gut lda = decomposition.LatentDirichletAllocation(doc_topic_prior=0.000000000000000001, topic_word_prior=0.00000001,n_components=25,max_iter=1000,learning_method='online',learning_offset=50,n_jobs=1,random_state=111)



lda = decomposition.LatentDirichletAllocation(doc_topic_prior=0.0000000000000000001, topic_word_prior=0.00000001,n_components=20,max_iter=300,learning_method='online',learning_offset=50,n_jobs=1,random_state=111)

W1 = lda.fit_transform(df)
#H1 = lda.components_

In [785]:
#Help Print die Topics
def print_topics(model, count_vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [786]:
# Wie viele Topics und wörter
number_topics = 50
number_words = 5


In [787]:
print("Topics found via LDA:")
print_topics(lda,vectorizer, number_words)

Topics found via LDA:

Topic #0:
flares feet legs spinal nerve

Topic #1:
issues rash stretches stretch lymph

Topic #2:
tremor spine exhaustion chest tremor leg

Topic #3:
chronic lyme pressure disease blood

Topic #4:
terrible mouth liver breath dairy

Topic #5:
nerves arms depression symptom attack

Topic #6:
eyes problem bladder cold myoclonic

Topic #7:
headaches rrms lot legs nails

Topic #8:
leg arm left weakness symptoms

Topic #9:
spasticity lesions night side dropfoot

Topic #10:
fatigue foot sensations weakness insomnia

Topic #11:
symptoms eye numbness andor teeth

Topic #12:
atrophy migraines muscle breathing knee

Topic #13:
worse vision disability brain stiffness

Topic #14:
problems stomach psychosis psoriasis ears

Topic #15:
legs hearing wrist epilepsy ppms

Topic #16:
problems illness symptoms legs moods

Topic #17:
flare pains aches fmla dystonia

Topic #18:
hand good side anger lipoatrophy

Topic #19:
pain spasms issues sleep muscle


In [788]:
#https://www.programmersought.com/article/6566770914/
# guthttps://www.youtube.com/watch?v=SF50IK5XgKA
# pyldavis sortiert topics neu ohne vorwarnung https://github.com/bmabey/pyLDAvis/issues/127
import pyLDAvis
import pyLDAvis.sklearn


vis = pyLDAvis.sklearn.prepare(lda, df, vectorizer, sort_topics=True)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.100673  0.006511       1        1  22.936244
8      0.029226 -0.103902       2        1   9.325946
9      0.078769  0.253729       3        1   8.310249
10     0.212814 -0.117830       4        1   7.940905
11    -0.003372 -0.197505       5        1   7.479224
5     -0.027794 -0.088064       6        1   4.524469
0     -0.128655  0.025692       7        1   4.432133
13     0.172046 -0.169993       8        1   4.118235
17     0.033178 -0.021333       9        1   3.601108
14    -0.201975  0.056274      10        1   3.508772
7     -0.127255 -0.065654      11        1   3.324100
6      0.128706  0.000029      12        1   3.231764
3      0.103349  0.084507      13        1   3.231764
16    -0.287590 -0.131808      14        1   3.139428
18     0.082766  0.069018      15        1   2.585411
12    -0.132376  0.068770      16        1   2.400739
1      0.167879 -0.076952      17        1   1.846722
15    -0.216812 -0.019946      18        1   1.569714
4     -0.004493  0.117764      19        1   1.292705
2      0.020917  0.310693      20        1   1.200369, topic_info=         Term          Freq      Total Category  logprob  loglift
368      pain  6.800000e+01  68.000000  Default  30.0000  30.0000
131   fatigue  2.600000e+01  26.000000  Default  29.0000  29.0000
244       leg  1.800000e+01  18.000000  Default  28.0000  28.0000
255      legs  1.400000e+01  14.000000  Default  27.0000  27.0000
526  symptoms  2.400000e+01  24.000000  Default  26.0000  26.0000
..        ...           ...        ...      ...      ...      ...
131   fatigue  4.212058e-08  26.510385  Topic20 -19.5477 -15.8377
192      hard  3.683885e-08   1.956671  Topic20 -19.6817 -13.3654
353     night  3.737782e-08   7.831824  Topic20 -19.6671 -14.7378
255      legs  3.740201e-08  14.453613  Topic20 -19.6665 -15.3499
13    anxiety  3.671717e-08   8.909167  Topic20 -19.6850 -14.8846

[783 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         3  0.799610         abdomen
1         9  0.920283           aches
2         9  1.051293     aches pains
3         3  1.034301          active
4         3  1.034301  active lesions
...     ...       ...             ...
578       8  1.022055   worse fatigue
579       9  1.051292     worse flare
580       8  1.015108      worse pump
581       1  0.998326           worst
582      18  0.804687           wrist

[638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 9, 10, 11, 12, 6, 1, 14, 18, 15, 8, 7, 4, 17, 19, 13, 2, 16, 5, 3])

In [604]:
# test gegen unseen data

In [605]:
vis.topic_info

,Term,Freq,Total,Category,logprob,loglift
368,pain,68.000000,68.000000,Default,30.0000,30.0000
131,fatigue,26.000000,26.000000,Default,29.0000,29.0000
244,leg,18.000000,18.000000,Default,28.0000,28.0000
255,legs,14.000000,14.000000,Default,27.0000,27.0000
526,symptoms,24.000000,24.000000,Default,26.0000,26.0000
...,...,...,...,...,...,...
255,legs,0.000001,14.453264,Topic20,-16.3612,-12.0446
353,night,0.000001,7.831776,Topic20,-16.3612,-11.4319
118,extreme,0.000001,1.960796,Topic20,-16.3614,-10.0473
192,hard,0.000001,1.956673,Topic20,-16.3617,-10.0455
